In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from keras.models import Sequential, load_model, Model
from keras.layers import *
from keras import optimizers
from keras import utils
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
import keras

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneGroupOut
from sklearn.utils import shuffle 

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

import pickle as pkl
import h5py

from pathlib import Path
import os.path
import sys
import datetime
import time



Using TensorFlow backend.


In [2]:
import telegram
from keras.callbacks import Callback
from callbacks import TelegramCallback
from callbacks.TelegramData import TelegramData


# create callback
config = {
    'token': TelegramData.Token,   # paste your bot token
    'telegram_id': TelegramData.ID,                                   # paste your telegram_id
    'model_name': 'JAN_LSTM_CNN'
}

tg_callback = TelegramCallback(config)

In [3]:
class LoggingTensorBoard(TensorBoard):    

    def __init__(self, log_dir, settings_str_to_log, **kwargs):
        super(LoggingTensorBoard, self).__init__(log_dir, **kwargs)

        self.settings_str = settings_str_to_log

    def on_train_begin(self, logs=None):
        TensorBoard.on_train_begin(self, logs=logs)

        tensor =  tf.convert_to_tensor(self.settings_str)
        summary = tf.summary.text ("Run_Settings", tensor)

        with  tf.Session() as sess:
            s = sess.run(summary)
            self.writer.add_summary(s)

In [4]:
train_ids = [1,2,3, 7, 8, 9, 10,12,13,14,15,16,17]
test_ids = [4,5,6,11]

In [5]:
dfAll = pd.read_pickle("PklData/df_lstm_norm30.pkl")
dfCNN =  pd.read_pickle("PklData/df_blobs.pkl")

In [6]:
def f(row):
    if row['TaskID'] < 17:
        #val = "Knuckle"
        val = 0
    elif row['TaskID'] >= 17:
        #val = "Finger"
        val = 1
    return val
dfAll['InputMethod'] = dfAll.apply(f, axis=1)
dfAll.TaskID = dfAll.TaskID % 17

In [7]:
dfAll.head()

,userID,TaskID,VersionID,Blobs,BlobCount,GestureOnly,InputMethod
0,1,0,3,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",30,0,0
2,1,0,6,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 239,...",30,0,0
3,1,0,7,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",30,0,0
4,1,0,8,"[[[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0...",30,0,0
5,1,0,9,"[[[0, 0, 0, 0, 0, 1, 1, 0, 153, 181, 0, 0, 0, ...",30,0,0


In [8]:
#dfNew = dfAll[dfAll.TaskID.isin([0, 2, 4,6,8,9,10,11,13,15])].copy()
#y = pd.Series([0, 2, 4,6,8,9,10,11,13,15], index=[0,1,2,3,4,5,6,7,8,9])
#dfNew.TaskID = dfNew.TaskID.replace([0, 2, 4,6,8,9,10,11,13,15], [0,1,2,3,4,5,6,7,8,9])
#dfAll = dfNew

In [33]:
df_train = dfAll[dfAll.userID.isin(train_ids)]
df_test = dfAll[dfAll.userID.isin(test_ids)]

df_train2 = df_train[['Blobs', 'TaskID', 'InputMethod']].copy()
df_test2 = df_test[['Blobs', 'TaskID', 'InputMethod']].copy()

In [35]:
df_train2.head()

,Blobs,TaskID,InputMethod
0,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",0,0
2,"[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 239,...",0,0
3,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0,0
4,"[[[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0...",0,0
5,"[[[0, 0, 0, 0, 0, 1, 1, 0, 153, 181, 0, 0, 0, ...",0,0


In [36]:
x_train = np.concatenate(df_train2.Blobs.values).reshape(-1,30,27,15,1)
x_test = np.concatenate(df_test2.Blobs.values).reshape(-1,30,27,15,1)

y_train = df_train2.TaskID.values
y_test = df_test2.TaskID.values

y_train_cnn = df_train2.InputMethod.values
y_test_cnn = df_train2.InputMethod.values



In [15]:
# convert class vectors to binary class matrices (one-hot notation)
num_classes = len(dfAll.TaskID.unique()) 
y_train_one_hot = utils.to_categorical(df_train2.TaskID, num_classes)
y_test_one_hot = utils.to_categorical(df_test2.TaskID, num_classes)

In [37]:
# convert class vectors to binary class matrices (one-hot notation)
num_classes_cnn = 2
y_train_one_hot_cnn = utils.to_categorical(df_train2.InputMethod, num_classes_cnn)
y_test_one_hot_cnn = utils.to_categorical(df_test2.InputMethod, num_classes_cnn)

In [40]:
print(len(y_train_one_hot_cnn))
len(y_train_one_hot_cnn)

6125


6125

In [18]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
#config = tf.ConfigProto(device_count = {"GPU": 1})


# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.3

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

In [27]:
KnuckelFinger = load_model("10_01_19.h5", compile=True)
KnuckelFinger.summary()

#hist = KnuckelFinger.fit(x_test3, y_test_one_hot_cnn)

#a = KnuckelFinger.predict_classes(x_train3)
#asdf = x_train3[144].reshape((1,27,15,1))
#KnuckelFinger.predict_classes(asdf)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 27, 15, 64)        640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 27, 15, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 27, 15, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 8, 64)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 8, 64)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 8, 32)         18464     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 8, 32)         9248      
__________

In [41]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.utils import plot_model
from keras.models import save_model, load_model
import pydot

batch_size = 60
epochs = 100
timesteps = 30
data_dim = (27,15)




tf.get_default_graph()
model = Sequential()

#### Gesture structure
A0 = Input(shape=(30,27,15,1),name='A0')
A1 = TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', 
                            kernel_regularizer=regularizers.l2(0.01)),input_shape=(30, 27,15,1), name='A1')(A0)
A2 = TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),name='A2')(A1)
#model.add(TimeDistributed(BatchNormalization(axis=-1)))
A3 = TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, padding='same', data_format='channels_last'),name='A3')(A2)
A4 = TimeDistributed(Dropout(0.20), name='A4')(A3)


A5 = TimeDistributed(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),name='A5')(A4)
A6 = TimeDistributed(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),name='A6')(A5)
#model.add(TimeDistributed(BatchNormalization(axis=-1)))
A7 = TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, padding='same', data_format='channels_last'),name='A7')(A6)
A8 = TimeDistributed(Dropout(0.10), name='A8')(A7)

A9 = TimeDistributed(Flatten(), name='A9')(A8)

A10 = LSTM(128, return_sequences=True, input_shape=(timesteps, data_dim), name='A10', kernel_regularizer=regularizers.l2(0.01))(A9)
A11 = LSTM(64, kernel_regularizer=regularizers.l2(0.01), name='A11')(A10)
A12 = Dense(num_classes, activation='softmax', name='A12')(A11)
A = Model(inputs=A0, outputs=A12)
# Output is a (17,1) tensor which tells us about finger or knuckle input

### END Gesture structure

#print(KnuckelFinger.outputs[0])
model = Model(inputs = KnuckelFinger.layers[0].input, outputs = KnuckelFinger.layers[-1].output)
B0 = TimeDistributed(model, name='B0')(A0)
B1 = Flatten(name="B1")(B0)
B2 = Dense(2, activation='softmax', name='B2')(B1)
#B1 = Reshape((27,15,-1),name='B1')(A0)

#B2 = Conv2D(64 ,kernel_size=(3,3), input_shape=(30,27,15,1),activation='relu',name='B2')(B1)
#B3 = Flatten(name='B3')(B2)
#B3 = Dense(128, activation='relu', kernel_regularizer=keras.regularizers.L1L2(0.02, 0.15),name='B3', use_bias=True)(B2)
#B3 = Dense(2, activation='softmax',name='B4')(B0)
# Output is a (2,1) tensor which tells us about finger or knuckle input
merged = Model(inputs=[A0],outputs=[A12, B2])


#merged.add_weight(KnuckelFinger, shape=(27,15))
merged.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
A0 (InputLayer)                 (None, 30, 27, 15, 1 0                                            
__________________________________________________________________________________________________
A1 (TimeDistributed)            (None, 30, 27, 15, 6 640         A0[0][0]                         
__________________________________________________________________________________________________
A2 (TimeDistributed)            (None, 30, 27, 15, 6 36928       A1[0][0]                         
__________________________________________________________________________________________________
A3 (TimeDistributed)            (None, 30, 14, 8, 64 0           A2[0][0]                         
__________________________________________________________________________________________________
A4 (TimeDi

In [42]:
print(x_train.shape)
print(y_train_one_hot.shape)
print(y_train_one_hot_cnn.reshape(6125,2))

(6125, 30, 27, 15, 1)
(6125, 17)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [ ]:
optimizer = optimizers.Adam(lr = 0.0001, decay=1e-6)
#optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.1)
merged.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
history = merged.fit(x_train, [y_train_one_hot, y_train_one_hot_cnn],
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, [y_test_one_hot, y_test_one_hot_cnn]))
                        #callbacks=[storer,logger,tg_callback, learning_rate_reduction])

#plot_model(merged,to_file='demo.png',show_shapes=True)

Train on 6125 samples, validate on 1963 samples
Epoch 1/100
1680/6125 [=======>......................] - ETA: 59s - loss: 13.0451 - A12_loss: 2.7465 - B2_loss: 0.6451 - A12_acc: 0.1887 - B2_acc: 0.6208 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['val_acc'], label="Test Accuracy")
plt.plot(history.history['acc'], label="Training Accuracy")
plt.legend()


In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    model.save('lstm_23_01_19.h5')